In [1]:
.libPaths("/home//amazigh.mokhtari//NeuroDev_ADD/R/r_3.6.0/")
#library (argparser)
library (r.jive)
library (dplyr)
library(corrplot)



Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


corrplot 0.84 loaded



In [2]:
omics = readRDS("~/NeuroDev_ADD/Psy/CT_Scar/Analysis/CT-Scar/00_RGCCA/Momix/##Male_Female_Supervised/00_Data/Female_MDD_omic_miRNAmRNADNAm.RDS")


In [3]:
covFEMALE = readRDS ("/home//amazigh.mokhtari/NeuroDev_ADD/Psy/CT_Scar/Analysis/CT-Scar/00_RGCCA/Momix/Female_MDD/00_Data/cov.icm.RDS")
rownames(covFEMALE) = gsub( "_L[0-9]+", "" ,   rownames(covFEMALE)  )
covFEMALE = covFEMALE [ rownames(omics$miRNA) , ]
Group = as.numeric ( as.factor (covFEMALE$Group)) 

In [10]:
boot_rnames = list()
covFEMALE = covFEMALE %>% 
  mutate(s_names = rownames(.)) 
boot_rnames = lapply (1:500 , function (x) {
        covFEMALE %>%
          group_by(Group) %>% 
          sample_n(nrow(.), replace=T) %>% .$s_names %>% unlist })

In [5]:
#########################
#########################
#########################

#### Function to extract Jive Results
Jive_extract = function (JIVE_dry) {
    ### Extract Jive Results 
    rankJV   = JIVE_dry$rankJ
    rankIV.v = JIVE_dry$rankA   

    J  = numeric(0)
    ng = 0

    factors_jive = features_w_jive = list()

    for(j in 1:length(JIVE_dry$data)){

        J  = rbind(J,JIVE_dry$joint[[j]]);
        ng = c(ng,dim(JIVE_dry$joint[[j]])[1])
      }

    svd.o = svd(J);
    jV    = svd.o$v %*% diag(svd.o$d);

    for(j in 1:length(JIVE_dry$data)){
        features_w_jive[[j]]  =  svd.o$u[(1+sum(ng[1:j])):sum(ng[1:j+1]),1:rankJV]; ###error in dimension
        rownames(features_w_jive[[j]]) = colnames(omics [[names(JIVE_dry$data[j] )]])   #rownames( t(JIVE_dry$data[[j]]))
        colnames(features_w_jive[[j]]) = paste0("JIVE_" , 1:num_factors )
      }

    factors_jive = jV[,1:rankJV]
    colnames(factors_jive) = paste0("JIVE_" , 1:10 )


    return (list( factors_jive = factors_jive, features_w_jive = features_w_jive )) }

########################

cor.mtest = function(mat, ...) {
    mat = as.matrix(mat)
    n = ncol(mat)
    p.mat = matrix(NA, n, n)
    diag(p.mat) <- 0
    for (i in 1:(n - 1)) {
        for (j in (i + 1):n) {
            tmp = cor.test(mat[, i], mat[, j], ...)
            p.mat[i, j] = p.mat[j, i] <- tmp$p.value
        }
    }
  colnames(p.mat) = rownames(p.mat) <- colnames(mat)
  p.mat
}



cor_mat = function (x){
    M = cor(x)
p.mat = cor.mtest(x)
col = colorRampPalette(c("#BB4444", "#EE9988", "#FFFFFF", "#77AADD", "#4477AA"))
plot = corrplot(M, method="color", col=col(200),  
         type="upper", 
         addCoef.col = "black", # Ajout du coefficient de corrélation
         tl.col="black", tl.srt=45, #Rotation des etiquettes de textes
         # Combiner avec le niveau de significativité
         p.mat = p.mat, sig.level = 0.05, insig = "blank", 
         # Cacher les coefficients de corrélation sur la diagonale
         diag=FALSE 
         )
# get top fact cor to Group 
    pval_group = p.mat [ 1:nrow(p.mat)-1, ncol(p.mat)] 
    
    name_fact_ = names(which.min(pval_group [ pval_group < 0.05]))
    if (length(name_fact_) != 0 ) {
         top.fact =  name_fact_   
    }
    
    return( list (cor.mat = M , p.mat = p.mat , plot = plot , top.fact = top.fact))
}


#### Get top features 


get_top_feature = function (features_w , top_cor_ ) {
    ### remove the Cov table if exist    
    lapply (1:length(features_w) , function (x) {   X =  features_w [[x]] %>% 
                         .[order ( - abs(.[,  top_cor_  ])  ) ,] ; # order the features according to the top factor that corr the most with phenotype
                                if ( grepl("miR" , rownames(X) [1] ) ){ ### if omic == miRNA %>% get the top 2% (~ 14 miRNA)
                                    X %>% head (n = round (nrow(.) *0.02 )  ) %>% 
                                    rownames
                                }else{
                                    X %>% head (n = round (nrow(.) *0.1 )  ) %>%  ### if omic is else  %>% get the top 10% 
                                    rownames
                                                                                                              }
                                                                                                              } )  

}

In [6]:
library(foreach)
library(doParallel)

cl <- parallel::makeCluster(4)
doParallel::registerDoParallel(cl)

Loading required package: iterators

Loading required package: parallel



In [11]:
num_factors = 10 
foreach(i = 1:500)  %do% {
op = lapply (omics  , function (x) x [ boot_rnames [[i]] , ]     )
factorizations_jive = jive( lapply (op,t) , 
                            rankJ   = num_factors, 
                            rankA   = rep ( num_factors, length( op ) ), 
                            method  = "given", 
                            conv    = "default", 
                            maxiter = 1000, 
                            showProgress = FALSE)

JIVE_Fact   =  Jive_extract(factorizations_jive) [["factors_jive"]] 

features_w_jive =   Jive_extract(factorizations_jive) [["features_w_jive"]]   
features_w_jive = features_w_jive[1:(length(features_w_jive)-1) ]


cor_meth_jive = cor.mtest (cbind(JIVE_Fact , Group )) [1:10 , 11]
top_cor_jive = names(cor_meth_jive ) [which.min(cor_meth_jive)] 

Top_feature = get_top_feature (features_w_jive , top_cor_jive )
write.table (Top_feature[[1]] , file = paste0("boots/miRNA_jive_",i,  top_cor_jive ,".txt") ,
                                 sep = "\t" ,
                                 row.names = F ,
                                 col.names = F ,
                                 quote = F)
write.table (Top_feature[[2]] , file = paste0("boots/mRNA_jive_",i,  top_cor_jive ,".txt") ,
                                 sep = "\t" ,
                                 row.names = F ,
                                 col.names = F ,
                                 quote = F)
write.table (Top_feature[[3]] , file = paste0("boots/DNAm_jive_",i,  top_cor_jive ,".txt") ,
                                 sep = "\t" ,
                                 row.names = F ,
                                 col.names = F ,
                                 quote = F)
             }

Timing stopped at: 0.223 0.195 0.036



ERROR: Error in {: task 64 failed - "Error: svd(x) and svd(t(x)) both failed to converge."


In [12]:
Files_mRNA = list.files("boots/", pattern=glob2rx ("mRNA_*.txt"), 
    full.names=FALSE)
Files_miRNA = list.files("boots/", pattern=glob2rx ("miRNA_*.txt"), 
    full.names=FALSE)
Files_DNAm = list.files("boots/", pattern=glob2rx ("DNAm_*.txt"), 
    full.names=FALSE)

In [13]:
df_mRNA  =  lapply (Files_mRNA ,function (x) read.csv2(file = file.path("boots/", x) , sep="\t", col.names = F) )
df_miRNA =  lapply (Files_miRNA ,function (x) read.csv2(file = file.path("boots/", x) , sep="\t", col.names = F) )
df_DNAm  =  lapply (Files_DNAm ,function (x) read.csv2(file = file.path("boots/", x) , sep="\t", col.names = F) )

In [26]:
boots_mRNA = df_mRNA %>% unlist %>% table %>% as.data.frame %>% arrange (-Freq) %>% mutate (per = Freq * 100 / 500)
boots_miRNA = df_miRNA %>% unlist %>% table %>% as.data.frame %>% arrange (-Freq) %>% mutate (per = Freq * 100 / 500)
boots_DNAm = df_DNAm %>% unlist %>% table %>% as.data.frame %>% arrange (-Freq) %>% mutate (per = Freq * 100 / 500)

In [32]:
boots_list = list (boots_mRNA = boots_mRNA , 
boots_miRNA = boots_miRNA  , 
boots_DNAm = boots_DNAm)